In [1]:
from scipy.spatial import cKDTree

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()

from neuprint import Client, fetch_neurons, fetch_skeleton, NeuronCriteria as NC, fetch_synapses, SynapseCriteria as SC

Loading BokehJS ...

In [2]:
c = Client('neuprint.janelia.org', 'hemibrain:v1.2.1')

In [3]:
#neurons, syn_counts = fetch_neurons(NC(status='Traced', cropped=False))

### Fetch the skeleton and synapses for a body

In [4]:
body = 1136399017
synapses = fetch_synapses(body)
synapses.head()

,bodyId,type,roi,x,y,z,confidence
0,1136399017,pre,AOTU(R),12049,32632,14338,0.957
1,1136399017,pre,AOTU(R),11922,32334,13794,0.979
2,1136399017,pre,AOTU(R),11871,32389,13778,0.997
3,1136399017,pre,AOTU(R),11866,32376,13731,0.991
4,1136399017,pre,AOTU(R),11834,32363,13751,0.969


In [5]:
skeleton = fetch_skeleton(body, heal=True)
skeleton.head()

,rowId,x,y,z,radius,link
0,1,12798.000000,30268.000000,15812.000000,21.000000,-1
1,2,12746.700195,30370.699219,15788.700195,55.464100,1
2,3,12727.200195,30411.199219,15767.599609,68.081200,2
3,4,12705.299805,30475.599609,15716.400391,58.952702,3
4,5,12687.400391,30499.500000,15692.500000,50.619999,4


### Associate each synapse with it's nearest skeleton node (rowId)

In [6]:
kd = cKDTree(skeleton[[*'xyz']].values)
distances, indexes = kd.query(synapses[[*'xyz']].values)
synapses['link'] = skeleton.loc[indexes, 'rowId'].values

### Append synapses as nodes to the skeleton

In [7]:
skeleton['structure'] = 'neurite'
synapses['rowId'] = synapses.index + skeleton['rowId'].max() + 1
synapses['structure'] = synapses['type']
synapses['radius'] = 0.0

final_skeleton = pd.concat((skeleton, synapses[['rowId', *'xyz', 'radius', 'link', 'structure']]))
final_skeleton

,rowId,x,y,z,radius,link,structure
0,1,12798.000000,30268.000000,15812.000000,21.000000,-1,neurite
1,2,12746.700195,30370.699219,15788.700195,55.464100,1,neurite
2,3,12727.200195,30411.199219,15767.599609,68.081200,2,neurite
3,4,12705.299805,30475.599609,15716.400391,58.952702,3,neurite
4,5,12687.400391,30499.500000,15692.500000,50.619999,4,neurite
...,...,...,...,...,...,...,...
358,2033,12073.000000,32575.000000,14386.000000,0.000000,651,post
359,2034,10072.000000,32572.000000,14464.000000,0.000000,685,post
360,2035,10647.000000,31797.000000,14057.000000,0.000000,760,post
361,2036,11203.000000,30673.000000,14839.000000,0.000000,1086,post


### Plot the new skeleton

In [8]:
# Join parent/child nodes for plotting as line segments below.
# (Using each row's 'link' (parent) ID, find the row with matching rowId.)
segments = final_skeleton.merge(final_skeleton, 'inner',
                               left_on='link',
                               right_on='rowId',
                               suffixes=['_child', '_parent'])

colors = {
    'neurite': 'green',
    'pre': 'blue',
    'post': 'red'
}
segments['color'] = segments['structure_child'].map(colors)

In [9]:
segments.head()

,rowId_child,x_child,y_child,z_child,radius_child,link_child,structure_child,rowId_parent,x_parent,y_parent,z_parent,radius_parent,link_parent,structure_parent,color
0,2,12746.700195,30370.699219,15788.700195,55.4641,1,neurite,1,12798.000000,30268.000000,15812.000000,21.0000,-1,neurite,green
1,3,12727.200195,30411.199219,15767.599609,68.0812,2,neurite,2,12746.700195,30370.699219,15788.700195,55.4641,1,neurite,green
2,1156,12812.000000,30436.000000,15868.000000,12.7990,2,neurite,2,12746.700195,30370.699219,15788.700195,55.4641,1,neurite,green
3,1753,12714.000000,30360.000000,15783.000000,0.0000,2,pre,2,12746.700195,30370.699219,15788.700195,55.4641,1,neurite,blue
4,1795,12775.000000,30358.000000,15756.000000,0.0000,2,post,2,12746.700195,30370.699219,15788.700195,55.4641,1,neurite,red


In [12]:
p = figure()
p.y_range.flipped = True

# Plot skeleton segments (in 2D)
p.segment(x0='x_child', x1='x_parent',
          y0='z_child', y1='z_parent',
          line_color='color',
          line_width=2,
          source=segments)

syn_segments = segments.query('structure_child != "neurite"')
p.scatter(syn_segments['x_child'], syn_segments['z_child'], color=syn_segments['color'], width=5)

show(p)